In [10]:
## Dependencies

import pandas as pd
import pathlib as path

import requests
import json
from pprint import pprint

import numpy as np

from io import StringIO

# to read the ZIP file
import requests
import zipfile
import io



# (1) create DF for countries

#### information will be used to merge or generate vizualisations
source : https://api.worldbank.org/v2/country?format=json

In [11]:
## retrieve all the countries from world bank API

### generate DF Countries

page = 1   ## there are 6 pages in the json file
data = []
url_countries = 'https://api.worldbank.org/v2/country?format=json'

for page in range(1,7):
    response = requests.get(f"{url_countries}&page={page}")
    json_data = response.json()
    
    # Add the data from the current page
    data.extend(json_data[1])  
    
## create DF and select columns

countries_wb_df = pd.DataFrame(data)
countries_wb_df['region'] = countries_wb_df['region'].apply(lambda x: x['value'])
countries_wb_df = countries_wb_df[['id', 'iso2Code', 'name', 'region','capitalCity', 'longitude', 'latitude']]
countries_wb_df = countries_wb_df.rename(columns={
                                                    'name':'country',
                                                    'id':'iso3Code'
                                                    })


regions = (countries_wb_df['region'] != "Aggregates")
countries_wb_df = countries_wb_df[regions]


countries_wb_df.head()

,iso3Code,iso2Code,country,region,capitalCity,longitude,latitude
0,ABW,AW,Aruba,Latin America & Caribbean,Oranjestad,-70.0167,12.5167
2,AFG,AF,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Kabul,69.1761,34.5228
5,AGO,AO,Angola,Sub-Saharan Africa,Luanda,13.242,-8.81155
6,ALB,AL,Albania,Europe & Central Asia,Tirane,19.8172,41.3317
7,AND,AD,Andorra,Europe & Central Asia,Andorra la Vella,1.5218,42.5075


In [12]:
path_raw_data = '../Output/countries_immigration.csv'

In [13]:
### Export to csv file in output as RAW DATA

countries_wb_df.to_csv(path_raw_data, index=False)

### import continent assignement

In [14]:
countries_wb_df = pd.read_csv(path_raw_data)

In [15]:
countries_wb_df.head()

,iso3Code,iso2Code,country,region,capitalCity,longitude,latitude
0,ABW,AW,Aruba,Latin America & Caribbean,Oranjestad,-70.0167,12.51670
1,AFG,AF,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Kabul,69.1761,34.52280
2,AGO,AO,Angola,Sub-Saharan Africa,Luanda,13.2420,-8.81155
3,ALB,AL,Albania,Europe & Central Asia,Tirane,19.8172,41.33170
4,AND,AD,Andorra,Europe & Central Asia,Andorra la Vella,1.5218,42.50750


In [16]:
path_continent = '../Input/Countries_continents_immigration.csv'
continents = pd.read_csv(path_continent)
continents.head()

,iso3Code,country,continent,sub-region,continent (DZ compare),sub-region (DZ compare),UMA,COMESA,CEA,CEEAC,CEDEAO,SADC,OCDE,UE,Euro,algeria
0,AFG,Afghanistan,Asia,Southern Asia,Asia,Southern Asia,no,no,no,no,no,no,no,no,no,no
1,ALB,Albania,Europe,Southern Europe,Europe,Southern Europe,no,no,no,no,no,no,no,no,no,no
2,DZA,Algeria,Africa,North Africa,Algeria,Algeria,yes,no,no,no,no,no,no,no,no,yes
3,ASM,American Samoa,Oceania,NaN,Oceania,NaN,no,no,no,no,no,no,no,no,no,no
4,AND,Andorra,Europe,Southern Europe,Europe,Southern Europe,no,no,no,no,no,no,no,no,no,no


In [17]:
countries_df = pd.merge(countries_wb_df, continents, on='country')

countries_df = countries_df[['country', 'iso3Code_x', 'iso2Code',  'region', 'continent', 'sub-region', 'OCDE' , 'capitalCity',
       'longitude', 'latitude']]

countries_df = countries_df.rename(columns={'iso3Code_x' : 'iso3Code'})

countries_df['OCDE'] = countries_df['OCDE'].replace({'yes': 1, 'no': 0})


countries_df

C:\Users\nazim\AppData\Local\Temp\ipykernel_23896\2398932225.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  countries_df['OCDE'] = countries_df['OCDE'].replace({'yes': 1, 'no': 0})


,country,iso3Code,iso2Code,region,continent,sub-region,OCDE,capitalCity,longitude,latitude
0,Aruba,ABW,AW,Latin America & Caribbean,Latin America & Caribbean,NaN,0,Oranjestad,-70.0167,12.51670
1,Afghanistan,AFG,AF,"Middle East, North Africa, Afghanistan & Pakistan",Asia,Southern Asia,0,Kabul,69.1761,34.52280
2,Angola,AGO,AO,Sub-Saharan Africa,Africa,Central Africa,0,Luanda,13.2420,-8.81155
3,Albania,ALB,AL,Europe & Central Asia,Europe,Southern Europe,0,Tirane,19.8172,41.33170
4,Andorra,AND,AD,Europe & Central Asia,Europe,Southern Europe,0,Andorra la Vella,1.5218,42.50750
...,...,...,...,...,...,...,...,...,...,...
211,Kosovo,XKX,XK,Europe & Central Asia,Europe,Southern Europe,0,Pristina,20.9260,42.56500
212,"Yemen, Rep.",YEM,YE,"Middle East, North Africa, Afghanistan & Pakistan",Asia,Western Asia,0,Sana'a,44.2075,15.35200
213,South Africa,ZAF,ZA,Sub-Saharan Africa,Africa,Austral Africa,0,Pretoria,28.1871,-25.74600
214,Zambia,ZMB,ZM,Sub-Saharan Africa,Africa,Eastern Africa,0,Lusaka,28.2937,-15.39820


In [18]:
### Export to csv file in output

countries_df.to_csv('../Output/countries_immigration.csv', index=False)
